In [1]:
# Import libraries
import pandas as pd
from pandas.tseries.offsets import BDay
import pickle
from pathlib import Path
from datetime import date
from utils import *


In [2]:
# DATE CHANGES - IN IRA LIMIT FILE OFFSET TO BDAYS & DO THE SAME FOR ANY INCOMING INVESTMENT DATES

In [3]:
# This is the information to be fed from the application

today = date.today()

# # Dummy data for investment dates
# investment_dates = pd.DataFrame(
#     index=[yearly_contr],
#     data=[],
#     columns=['months_to_invest']
# )

#investment_dates.index = pd.to_datetime(investment_dates.index)


In [4]:
# Annual contribution at the beginning of each year 

yearly_contr = pd.date_range(start='2012-01-01', end = today, freq='BAS')

# Create df for investment dates
investment_dates = pd.DataFrame(
     index=yearly_contr,
     data=len(yearly_contr)*[12],
     columns=['months_to_invest']
)

#investment_dates = pd.DataFrame(all_ira_contr)
#investment_dates = investment_dates.rename(columns={'ira_limit':'months_to_invest'})

investment_dates.index
investment_dates.index = pd.to_datetime(investment_dates.index)
investment_dates

,months_to_invest
2012-01-02,12
2013-01-01,12
2014-01-01,12
2015-01-01,12
2016-01-01,12
2017-01-02,12
2018-01-01,12
2019-01-01,12
2020-01-01,12
2021-01-01,12


In [5]:
# Set the start date of the investment account
portfolio_investment_start_date = pd.Timestamp(investment_dates.index[0])

next_transaction = investment_dates[:].loc[investment_dates.months_to_invest>0, 'months_to_invest'].index[0]
next_transaction = pd.Timestamp(next_transaction)

f'The portfolio start datetime is {portfolio_investment_start_date} and the first investment date is {next_transaction}'


'The portfolio start datetime is 2012-01-02 00:00:00 and the first investment date is 2012-01-02 00:00:00'

In [6]:
# This function sets up the new holding account for the IRA contribution amount to be held until contribution date(s) 
# This function also creates an investment account with a portfolio attached for investment and rebalancing

# Select the portfolio to be invested in
# Options are fidelity_index_focused_models, betterment, rick_ferris_core and vanguard_total_stock_etf
portfolio_selection = 'fidelity_index_focused_models'

# Read in portfolio data from pkl file
pkl_file = open(Path(f"./Resources/{portfolio_selection}.pkl"), 'rb')
dataframe = pickle.load(pkl_file)
pkl_file.close()

# create portfolio allocation df for future rebalancing
portfolio_weights = dataframe

# create new investment account with the portfolio funds as columns
investment_account = pd.DataFrame(columns=dataframe.columns.values)

# Create a new holding account 
holding_account = 0

display(f'{portfolio_selection} has been attached to the new investment account.')
display('The portfolio asset allocation is:')
display(portfolio_weights)
display(f'The holding account balance is ${holding_account}.')

'fidelity_index_focused_models has been attached to the new investment account.'

'The portfolio asset allocation is:'

,FXAIX,FSMDX,FSSNX,FSGGX,FXNAX,FCONX,BIL
0,35,3,4,18,35,3,2


'The holding account balance is $0.'

In [7]:
# Read in ira contribution data from pkl file

pkl_file = open(Path(f"./Resources/ira_contribution_limits.pkl"), 'rb')
dataframe = pickle.load(pkl_file)
pkl_file.close()

all_ira_contr = dataframe

# Locate the contribution limit for the portfolio start date
year = str(portfolio_investment_start_date.year)
yearly_ira_contribution = dataframe.loc[year]['ira_limit'][0]

# Fund the holding account with the annual contribution for the first year
holding_account = yearly_ira_contribution

In [8]:
# Create a dataframe of portfolio returns for the selected portfolio

portfolio_returns = pd.DataFrame()

for key in investment_account.keys():
    # Create dataframe from CSV file
    df = pd.read_csv(f"./data/{key}.csv")

    # Drop columns and set date index for concat
    df = df[['Date','Close']].set_index('Date')
    df.index = pd.to_datetime(df.index)

    # Rename columns to Ticker
    df = df.rename(columns={'Close':key})
    
    # Change Close to return
    df = df.pct_change()
    
    # Concat to empty dataframe
    portfolio_returns = pd.concat([portfolio_returns, df], axis=1)
        
# drop na values 
portfolio_returns = portfolio_returns.dropna()

# review
display(portfolio_returns.head(2))
display(portfolio_returns.tail(2))


,FXAIX,FSMDX,FSSNX,FSGGX,FXNAX,FCONX,BIL
Date,,,,,,,
2011-09-12,0.006826,0.005139,0.008247,-0.021627,-0.002532,0.0,0.000000
2011-09-13,0.009443,0.014315,0.018405,0.007368,-0.001692,0.0,-0.000218


,FXAIX,FSMDX,FSSNX,FSGGX,FXNAX,FCONX,BIL
Date,,,,,,,
2022-09-07,0.018380,0.023345,0.022042,0.006584,0.005753,0.0,0.000109
2022-09-08,0.006714,0.010097,0.008363,0.001635,-0.003813,0.0,0.000219


In [9]:
# Define transaction function

def transact(next_transaction):
    global holding_account
    try:       
        # Calculate the transaction amount (number of months to invest multiplied by annual ira contribution, divided equally into months)
        trans_amt = round(investment_dates.loc[next_transaction]['months_to_invest'] * (ira_contribution / 12), 2)
        debug_print(f'ira contr: {ira_contribution}')
        debug_print(f'transaction amount: {trans_amt}')
        
        # Validate investment amount and update holding account balance 
        debug_print(f'holding account: {holding_account}')
        if trans_amt <= holding_account:
            # update holding account balance
            holding_account = holding_account - trans_amt
            validated_amount = trans_amt
        
        else: 
            validated_amount = holding_account
            holding_account = 0
        return validated_amount
        debug_print(f'validated amount: {validated_amount}')
            
            ## Add zero option to hold rebalance

        # Divide the amount by the portfolio weightings and apply to investment account
        try:
            investment_account.loc[next_transaction] = (investment_account.iloc[-1].sum() + validated_amount) * portfolio_weights.iloc[0].values /100
        except:
            investment_account.loc[next_transaction] = validated_amount * portfolio_weights.iloc[0].values /100

        # Can these amounts be rounded to 2dp as they are loaded?

    # Update the last transaction date
    last_transaction_date = next_transaction

    day_after_last_trans_idx = investment_dates.index.get_loc(last_transaction_date) +1

    # Update the next transaction
    next_transaction = investment_dates[day_after_last_trans_idx:].loc[investment_dates.months_to_invest>0, 'months_to_invest'].index[0]
    next_transaction = pd.Timestamp(next_transaction)

    return next_transaction

    except:
        print('transaction does not exist')
        


SyntaxError: invalid syntax (3677235102.py, line 35)

In [ ]:
# Adjust the invested balance by the fund return until the next contribution

# Set parameters for range

# Set range
transaction_range = pd.date_range(
    start=pd.Timestamp(portfolio_investment_start_date), 
    end=today, 
    freq='B'
).tolist()

#transaction_range = [i.strftime('%Y-%m-%d') for i in transaction_range]   

# For each day in range, multiply by fund return
# loop of returns 
debug_print(f'The next transaction date at the beginning of the function is {next_transaction}')

for date in transaction_range:
    if date < portfolio_investment_start_date:
        continue
    
    if date < next_transaction:
        try:
            investment_account.loc[date] = investment_account.iloc[-1] * (1 + portfolio_returns.loc[date])
        except:
            pass
    else:
        transact(next_transaction)
        #returns next transaction
        print(next_transaction)
    
    


In [ ]:
# Maintenance of account
# append day by day performance
# trigger contribution dates to top up by investment_amount and rebalance

# This function aggregates the portfolio value by the fund return amount,
# unless a contribution is made.
# If a contribution is made, the amount is added to the portfolio and the portfolio is rebalanced back to target weights


# Note - freq 'MS' = month start

# create investment date range
inv_date_range = pd.date_range(
    start=portfolio_investment_start_date,
    end=today, 
    freq='B'
).tolist()

# pull dates from timestamp and convert to list
inv_date_range = [i.strftime('%Y-%m-%d') for i in inv_date_range]

# for each day from portfolio start until investment date
# append row, lookup fund return & multiply by shift(1)

transactions_df = pd.DataFrame(columns=['Amount'])

for day in inv_date_range:
    try:
        # Look up investment dates df and multiple number of months of investment by monthly ira contribution limit for the year
        if investment_dates.loc[day][0]:
            investment_amount = round(investment_dates.loc[day][0] * (ira_contribution / 12), 2)
            # Add to a dataframe
            df = pd.DataFrame(index = [day], data = [investment_amount], columns = ['Amount'])
            # append to the transactions df
            transactions_df = transactions_df.append(df, ignore_index=False)
    except:
        # If not an investment date, add row with zero
        df = pd.DataFrame(index = [day], data = [0], columns = ['Amount'])
        # append to the transactions df
        transactions_df = transactions_df.append(df, ignore_index=False)
        


In [ ]:


# # Set range
# zero_transaction_range = pd.date_range(
#     start=pd.Timestamp(last_transaction_date) + BDay(1), 
#     end=next_transaction, 
#     freq='B'
# ).tolist()

# zero_transaction_range = [i.strftime('%Y-%m-%d') for i in zero_transaction_range]

# # For each day in range, multiply by fund return
# # loop of returns 
# for date in zero_transaction_range:
#     try:
#         investment_account.loc[date] = investment_account.iloc[-1] * (1 + portfolio_returns.loc[date])
#     except:
#         pass


In [ ]:
#new_account.loc[len(new_account.index)] = new_account.iloc[-1] * (1 + portfolio_returns.loc[new_account.index[-1]])





In [ ]:
# IRA contribution limits
# create function to top up the holding account each year with the new limits